# Program Annotation

William Colgan May 15 2023

### Setup

In [33]:
library(useful)
library(tidyverse)
library(magrittr)
library(pheatmap)
library(gprofiler2)

### Load data

In [41]:
obs = read_csv("./data/K562_gwps_obs.csv")
var = read_csv("./data/K562_gwps_var.csv")
mat = read_csv("./data/K562_gwps_norm.csv",col_names = F)
mat <- mat %>% as.matrix()
colnames(mat) <- var$gene_name
rownames(mat) <- obs$gene_transcript %>% word(2,2,sep = "_")
mat = t(mat)
mat[mat > 5]  <-  5
mat[mat < -5]  <-  -5
temp  <- R.matlab::readMat("./output/interim/gwps-K=80-T=4.mat")
dict <- as.matrix(temp$D)
loading  <- as.matrix(temp$X)
colnames(loading)  <- colnames(mat)
#rownames(dict) <- rownames(mat)
rownames(dict)  <-  var$gene_id
names  <- c()
for (i in 1:nrow(loading)) {
  names  %<>% append(c(paste0("P",i)))
}
rownames(loading)  <- names
colnames(dict) <- names

Rows: 1946 Columns: 17
-- Column specification --------------------------------------------------------
Delimiter: ","
chr  (1): gene_transcript
dbl (15): UMI_count_unfiltered, num_cells_unfiltered, num_cells_filtered, co...
lgl  (1): core_control

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 3384 Columns: 10
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (2): gene_id, gene_name
dbl (7): mean, std, cv, gini, clean_mean, clean_std, clean_cv
lgl (1): in_matrix

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1946 Columns: 3384
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl (3384): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15...

i Use `spec(

In [97]:
dict  %>% rowSums()  %>% sort()  %>% tail()

ENSG00000135404 ENSG00000268621 ENSG00000158578 ENSG00000251562 ENSG00000167996 
       2.200579        2.207022        2.247766        2.276089        2.650123 
ENSG00000186652 
       3.786591

In [50]:
dict[,4] %>% sort() %>% head(10)

ENSG00000215021 ENSG00000123349 ENSG00000130208 ENSG00000137309 ENSG00000174444 
    -0.09962391     -0.06823151     -0.06488401     -0.05864981     -0.05745237 
ENSG00000136518 ENSG00000079246 ENSG00000131475 ENSG00000130203 ENSG00000026025 
    -0.05680444     -0.05628642     -0.05409580     -0.05379210     -0.05222049

In [59]:
dict[,10] %>% sort() %>% head(10)

ENSG00000089157 ENSG00000112306 ENSG00000170889 ENSG00000181163 ENSG00000108107 
    -0.09762059     -0.09565672     -0.07960092     -0.07956288     -0.07896535 
ENSG00000156508 ENSG00000175061 ENSG00000203875 ENSG00000137818 ENSG00000168028 
    -0.07806320     -0.07691729     -0.07092228     -0.06807501     -0.06786190

In [143]:
result  <- gost(abs(dict[,22]) %>% sort()  %>% rev() %>% head(30) %>% names(),ordered_query = T)
result$result  %>% arrange(p_value)  %>% filter(term_size < 1000) %>% head(10)

,query,significant,p_value,term_size,query_size,intersection_size,precision,recall,term_id,source,term_name,effective_domain_size,source_order,parents
,<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<list>
1,query_1,TRUE,0.000267562,11,26,4,0.1538462,0.36363636,HP:0012766,HP,Widened cerebral subarachnoid space,4851,9156,HP:0012704
2,query_1,TRUE,0.003592173,18,8,2,0.2500000,0.11111111,GO:0030992,GO:CC,intraciliary transport particle B,22122,1022,"GO:0030990, GO:0032991"
3,query_1,TRUE,0.007034564,25,8,2,0.2500000,0.08000000,GO:0030990,GO:CC,intraciliary transport particle,22122,1020,GO:0032991
4,query_1,TRUE,0.015332077,57,25,3,0.1200000,0.05263158,GO:0016655,GO:MF,"oxidoreductase activity, acting on NAD(P)H, quinone or similar compound as acceptor",20195,2896,GO:0016651
5,query_1,TRUE,0.016182094,1,1,1,1.0000000,1.00000000,GO:0046525,GO:MF,xylosylprotein 4-beta-galactosyltransferase activity,20195,6226,"GO:0035250, GO:0140103"
6,query_1,TRUE,0.020518807,10,26,3,0.1153846,0.30000000,HP:0007327,HP,Mixed demyelinating and axonal polyneuropathy,4851,4683,"HP:0009830, HP:0012447"
7,query_1,TRUE,0.020518807,10,26,3,0.1153846,0.30000000,HP:0001112,HP,Leber optic atrophy,4851,872,HP:0000587
8,query_1,TRUE,0.025077934,10,8,2,0.2500000,0.20000000,CORUM:6173,CORUM,Intraflagellar transport complex B core complex,3383,1947,CORUM:0000000
9,query_1,TRUE,0.026340129,48,8,2,0.2500000,0.04166667,GO:0097542,GO:CC,ciliary tip,22122,3061,"GO:0005929, GO:0110165"


In [32]:
install.packages("gprofiler2")


The downloaded binary packages are in
	/var/folders/qd/nvzjj6dd3bx9d0pqsk7m4tr80000gn/T//RtmpXb9x1z/downloaded_packages


In [40]:
var

gene_id,gene_name,mean,std,cv,in_matrix,gini,clean_mean,clean_std,clean_cv
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000419,DPM1,0.7850459,0.10545670,0.13433190,TRUE,0.07212226,0.8067765,0.10731252,0.13301395
ENSG00000001036,FUCA2,0.8603362,0.10897151,0.12666154,TRUE,0.06777804,0.8833603,0.11090380,0.12554765
ENSG00000001497,LAS1L,0.9034098,0.12070036,0.13360532,TRUE,0.07178739,0.9280114,0.12164642,0.13108289
ENSG00000001629,ANKIB1,0.1606149,0.03796154,0.23635125,TRUE,0.12764200,0.1644727,0.03915699,0.23807594
ENSG00000001630,CYP51A1,1.0960715,0.15919486,0.14524130,TRUE,0.07740526,1.1250014,0.16133223,0.14340624
ENSG00000002549,LAP3,0.7295924,0.10051703,0.13777147,TRUE,0.07399670,0.7499226,0.10223719,0.13633030
ENSG00000002586,CD99,1.4278524,0.16420257,0.11499968,TRUE,0.06137931,1.4668125,0.16711718,0.11393220
ENSG00000003402,CFLAR,0.8407919,0.11504367,0.13682777,TRUE,0.07145646,0.8624691,0.11902251,0.13800205
ENSG00000003436,TFPI,0.3138401,0.10325091,0.32899210,TRUE,0.15030165,0.3213279,0.10748815,0.33451230
